In [92]:
import pandas as pd
import os

In [93]:
# CARGA DE DATOS Y VISUALIZACIÓN DE NOMBRE DE COLUMNAS

df_frontur = pd.read_excel("../data/FRONTUR_DL.xlsx")
df_nacional = pd.read_excel("../data/TURISMO_INTERNO_PROV_CCAA_DL.xlsx")
df_gasto = pd.read_excel("../data/GASTO_TPV_DESTINO_CCAA_MES_DL.xlsx")

In [107]:
print(df_frontur.columns)

Index(['Unnamed: 0', 'AÑO', 'MES', 'PAIS_RESIDENCIA', 'TIPO_VISITANTE',
       'CCAA_DESTINO', 'VISITANTES'],
      dtype='object')


In [108]:
print(df_nacional.columns)

Index(['Unnamed: 0', 'AÑO', 'MES', 'CCAA_ORIGEN', 'PROVINCIA_ORIGEN',
       'CCAA_DESTINO', 'PROVINCIA_DESTINO', 'TURISTAS', 'PERNOCTACIONES',
       'ESTANCIA_MEDIA'],
      dtype='object')


In [109]:
print(df_gasto.columns)

Index(['Unnamed: 0', 'AÑO', 'MES', 'CCAA_DESTINO', 'TIPO_ORIGEN', 'GASTO',
       'TRANSACCION'],
      dtype='object')


In [97]:
# LIMPIEZA DE DATOS

def limpieza(df):
    df = df.dropna().copy()
    df.columns = df.columns.str.strip()

    for col in df.select_dtypes(include="object").columns:
        df.loc[:, col] = df[col].str.strip()

    if "AÑO" in df.columns:
        df.loc[:, "AÑO"] = df["AÑO"].astype(int)
    
    return df

df_frontur = limpieza(df_frontur)
df_nacional = limpieza(df_nacional)
df_gasto = limpieza(df_gasto)

In [98]:
# FUNCIONES REUTILIZABLES

def transformar_turismo_internacional(df):
    columnas_renombrar = {
        "AÑO": "Año",
        "CCAA_DESTINO": "CCAA",
        "VISITANTES": "Turistas_Internacionales"}
    df = df.rename(columns={k: v for k, v in columnas_renombrar.items() if k in df.columns})
    return df.groupby(["Año", "CCAA"], as_index=False)["Turistas_Internacionales"].sum()

In [99]:
def transformar_turismo_nacional(df):
    columnas_renombrar = {
        "AÑO": "Año",
        "CCAA_DESTINO": "CCAA",
        "TURISTAS": "Turistas_Nacionales" }
    df = df.rename(columns={k: v for k, v in columnas_renombrar.items() if k in df.columns})
    return df.groupby(["Año", "CCAA"], as_index=False)["Turistas_Nacionales"].sum()

In [100]:
def transformar_gasto(df):
    columnas_renombrar = {
        "AÑO": "Año",
        "CCAA_DESTINO": "CCAA",
        "GASTO": "Gasto" }
    df = df.rename(columns={k: v for k, v in columnas_renombrar.items() if k in df.columns})
    return df.groupby(["Año", "CCAA"], as_index=False)["Gasto"].sum()

In [101]:
# TRANSFORMACIÓN DE DATOS

df_frontur_grouped = transformar_turismo_internacional(df_frontur)
df_nacional_grouped = transformar_turismo_nacional(df_nacional)
df_gasto_grouped = transformar_gasto(df_gasto)

In [102]:
# UNIONES DE DATOS

df_internacional_gasto = pd.merge(df_frontur_grouped, df_gasto_grouped, on=["Año", "CCAA"], how="inner")
df_nacional_gasto = pd.merge(df_nacional_grouped, df_gasto_grouped, on=["Año", "CCAA"], how="inner")

In [105]:
# RESULTADOS

print("Turismo internacional + gasto:")
print(df_internacional_gasto.head(2))

Turismo internacional + gasto:
    Año       CCAA  Turistas_Internacionales  \
0  2019  Andalucía                  14652617   
1  2019     Aragón                    853190   

                                               Gasto  
0  18481196,7834039254,9233721507,9667760762,8818...  
1  503190,6810511516,071976147,6112487663,6852437...  


In [106]:
print("Turismo nacional + gasto:")
print(df_nacional_gasto.head(2))

Turismo nacional + gasto:
    Año       CCAA                                Turistas_Nacionales  \
0  2019  Andalucía  8910,03468,049626,02801,09247,020363,06920,083...   
1  2019     Aragón  666,0515,0975,0513,0303,01139,0614,0478,01326,...   

                                               Gasto  
0  18481196,7834039254,9233721507,9667760762,8818...  
1  503190,6810511516,071976147,6112487663,6852437...  
